In [1]:
import torch

## いくつかのフィルターの例

## CNNの計算

- 普通

In [71]:
conv2d_0 = torch.nn.Conv2d(1, 1, 3)
conv2d_0.weight.data[0] = 1
conv2d_0.bias.data =  torch.tensor([0.])
x = torch.ones(5,5).float()
conv2d_0(x.view(1, 1, 5, 5))

tensor([[[[9., 9., 9.],
          [9., 9., 9.],
          [9., 9., 9.]]]], grad_fn=<MkldnnConvolutionBackward>)

In [46]:
conv2d_0.weight

Parameter containing:
tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True)

In [49]:
conv2d_0.bias.data[0] = 0

In [52]:
conv2d_0.bias

Parameter containing:
tensor([0.0916], requires_grad=True)

- Stride

In [72]:
conv2d_s = torch.nn.Conv2d(1, 1, 3, stride=2)
conv2d_s.weight.data[0] = 1
conv2d_s.bias.data =  torch.tensor([0.])
x = torch.ones(5,5).float()
conv2d_s(x.view(1, 1, 5, 5))

tensor([[[[9., 9.],
          [9., 9.]]]], grad_fn=<MkldnnConvolutionBackward>)

- padding

In [78]:
conv2d = torch.nn.Conv2d(1, 1, 3, stride=2, padding=2, padding_mode='zeros')
conv2d.weight.data[0] = 1
conv2d.bias.data = torch.tensor([0.])
x = torch.ones(10,10).float()
conv2d(x.view(1, 1, 10, 10))

tensor([[[[1., 3., 3., 3., 3., 2.],
          [3., 9., 9., 9., 9., 6.],
          [3., 9., 9., 9., 9., 6.],
          [3., 9., 9., 9., 9., 6.],
          [3., 9., 9., 9., 9., 6.],
          [2., 6., 6., 6., 6., 4.]]]], grad_fn=<MkldnnConvolutionBackward>)

## Maxpooling